
#**Install Libraries**

In [1]:
!pip install groq
!pip install pytesseract
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr
!tesseract --version
!pip install pdfplumber pytesseract pdf2image
!sudo apt-get install -y tesseract-ocr
!pip install pytesseract pdf2image pdfminer.six
!pip install -U langchain-community
# !pip install pinecone-client


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [2]:
!pip install pinecone

#**Import Libraries**

In [4]:
# 1. Uninstall both versions to avoid conflict
!pip uninstall -y pinecone-client pinecone

# 2. Install the correct (latest) package
!pip install pinecone


Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
Found existing installation: pinecone 7.3.0
Uninstalling pinecone-7.3.0:
  Successfully uninstalled pinecone-7.3.0
  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)


In [5]:
import os
from pdf2image import convert_from_path
from pdfminer.high_level import extract_text
from PIL import Image
import pytesseract
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

from google.colab import files
from pinecone import Pinecone, ServerlessSpec
# from pinecone import Pinecone, ServerlessSpec, Index


from sentence_transformers import SentenceTransformer
from pdfminer.high_level import extract_text
from pdf2image import convert_from_path
import pytesseract
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import files

#**Generate Embeddings**


In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the model
model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')

# Encode a sample sentence
embedding = model.encode("Your text here")

# Check the shape of the embedding
print(f"Embedding shape: {np.shape(embedding)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Embedding shape: (1024,)


In [19]:


# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        text = extract_text(pdf_path)
        if text.strip():
            return text
    except Exception as e:
        print("Failed to extract text using pdfminer:", e)

    # Fallback to OCR if pdfminer fails
    print("Using Tesseract for OCR on scanned PDF...")
    images = convert_from_path(pdf_path)
    text = ""
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ+-*/=()[]{}^~<>πσ√'

    for i, image in enumerate(images):
        text += pytesseract.image_to_string(image, config=custom_config) + "\n"

    return text

# Upload PDF
uploaded = files.upload()  # Prompt for file upload
pdf_path = next(iter(uploaded.keys()))

# Extract text from the uploaded PDF
extracted_text = extract_text_from_pdf(pdf_path)

# Split the extracted text into chunks using LangChain
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.create_documents([extracted_text])

# Load Sentence-BERT model
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_embeddings(texts):
    return model.encode([text.page_content for text in texts])

# Generate embeddings for the chunks
embeddings = generate_embeddings(chunks)

# Print the embeddings
print("Generated Embeddings:")
for i, embedding in enumerate(embeddings):
    print(f"Chunk {i + 1}: {embedding[:10]}...")  # Print first 10 values of each embedding for brevity


Saving maths notes.pdf to maths notes.pdf
Generated Embeddings:
Chunk 1: [ 0.01922504 -0.01741229 -0.0289301   0.03117818  0.06268401  0.00260744
  0.00973056 -0.01882668  0.04315396  0.03709199]...
Chunk 2: [ 0.05813377 -0.0334365  -0.00977699 -0.01940453  0.01648323 -0.02271053
  0.01411506 -0.01813252 -0.00943714 -0.02474361]...
Chunk 3: [ 0.03718144 -0.0169562  -0.01037147 -0.00269452  0.02282565 -0.04582464
  0.0188677  -0.00773941  0.03102652 -0.0154209 ]...
Chunk 4: [ 0.05635441  0.02346014  0.00296076  0.00845108  0.03085265  0.00456765
 -0.00096389  0.02001291  0.01885663  0.02677421]...
Chunk 5: [ 0.03022064 -0.00223375 -0.0079095   0.00901241  0.03259278 -0.00207369
  0.00578639 -0.00477414 -0.009171   -0.0030172 ]...
Chunk 6: [ 0.05107099 -0.02453354 -0.03593611 -0.02105802  0.0291783  -0.03570003
 -0.001977    0.0450624   0.03093742 -0.0004896 ]...
Chunk 7: [ 0.04196399 -0.01261188  0.01857172 -0.06272508 -0.00534827 -0.01654642
 -0.01214598  0.0326718   0.01674577  0.0071

In [17]:
# Get the dimension of the embeddings
embedding_dimension = len(embeddings[0])
print(f"Embedding Dimension: {embedding_dimension}")

Embedding Dimension: 1024


In [18]:
 # Output: 1024


#**Pinecone Set-UP**

In [10]:
!pip install --upgrade pinecone

In [20]:
# Initialize Pinecone client
pc = Pinecone(api_key="")  # Use your actual API key

# List indexes
# indexes = pc.list_indexes().names()
# print("Indexes:", indexes)

# If an index exists, describe it, else create a new index
if indexes:
    index_name = indexes[0]  # Access the first index in the list and get the 'name'
    index_description = pc.describe_index(index_name)  # Pass the index name as a string
    print(f"Details of index '{index_name}':", index_description)
else:
    # Create a new index if none exist
    pc.create_index(
        name='chatbot',
        dimension=1024,  # Update dimension according to your embedding model
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Set your region here
        )
    )
    print("Created 'notes' index.")

# from pinecone import Pinecone

# Initialize Pinecone client (only once)
pc = Pinecone(api_key="your-pinecone-api-key")

# Access the 'chatbot' index properly
index = pc.Index(name="chatbot", host="https://chatbot-5kymish.svc.aped-4627-b74a.pinecone.io")

# Prepare data for insertion into Pinecone
vectors_to_insert = []
for i, embedding in enumerate(embeddings):
    vectors_to_insert.append((
        str(i),  # Unique ID for each vector
        embedding.tolist(),  # Embedding vector
        {"chunk_id": i, "content": chunks[i].page_content[:200]}  # Optional metadata
    ))

# Insert the embeddings into Pinecone
index.upsert(vectors=vectors_to_insert)

# Verify insertion by querying the Pinecone index (optional)
query_result = index.query(vector=embeddings[0].tolist(), top_k=3)  # Corrected to use keyword arguments
print("Query Result:", query_result)

# Optionally, check the status of the index
status = index.describe_index_stats()
print("Index Stats:", status)


Details of index 'adeen': {'deletion_protection': 'disabled',
 'dimension': 384,
 'host': 'adeen-5kymish.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'adeen',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'},
 'tags': None,
 'vector_type': 'dense'}
Query Result: {'matches': [{'id': '5', 'score': 0.187710494, 'values': []},
             {'id': '4', 'score': 0.145828873, 'values': []},
             {'id': '3', 'score': 0.140401378, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}
Index Stats: {'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 6}},
 'total_vector_count': 6,
 'vector_type': 'dense'}


In [22]:
import os
from groq import Groq
import numpy as np
from sentence_transformers import SentenceTransformer
# from pinecone import Index

# Set the API key for Groq
os.environ["GROQ_API_KEY"] = "your-groq-api-key"
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Initialize Pinecone client
pc = Pinecone(api_key="your-pinecone-api-key")  # Use your actual API key

# List indexes
indexes = pc.list_indexes().names()
# If an index exists, describe it, else create a new index
if indexes:
    index_name = indexes[0]  # Access the first index in the list and get the 'name'
    index_description = pc.describe_index(index_name)  # Pass the index name as a string
else:
    # Create a new index if none exist
    pc.create_index(
        name='chatbot',
        dimension=1024,  # Update dimension according to your embedding model
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Set your region here
        )
    )

# Access the 'notes' index using the Index class
pc = Pinecone(api_key="your-pinecone-api-key")

# Access the 'chatbot' index properly
index = pc.Index(name="chatbot", host="https://chatbot-5kymish.svc.aped-4627-b74a.pinecone.io")

# Load Sentence-BERT model for generating query embeddings
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')
# Function to generate embeddings for user input
def generate_embeddings(texts):
    return model.encode(texts)

# Function to get a response from Groq (LLaMA model)
def get_response_from_lama(prompt: str, model: str = "llama3-8b-8192") -> str:
    try:
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=model,
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        raise ValueError(f"Groq API call failed: {e}")

# Function to handle the entire process
def get_answer_from_query(user_query: str):
    # Step 1: Generate the query embedding
    query_embedding = generate_embeddings([user_query])[0]

    # Step 2: Search Pinecone for the most similar embeddings
    query_result = index.query(vector=query_embedding.tolist(), top_k=3, include_metadata=True)

    # Step 3: Collect relevant context from Pinecone
    context = "\n".join([match['metadata']['content'] for match in query_result['matches']])

    # Step 4: Create the prompt for the LLaMA model, including the context
    llama_prompt = f"Context:\n{context}\n\nUser Query: {user_query}\nAnswer:"

    # Step 5: Get the response from LLaMA using Groq
    response = get_response_from_lama(llama_prompt)

    return response

# User input query
user_query = input("Please enter your query: ")

# Get the answer from the Pinecone + LLaMA pipeline
answer = get_answer_from_query(user_query)

# Print only the question and the answer
print(f"Question: {user_query}")
print(f"Answer: {answer}")


Please enter your query: just tell me the definition of implicit functions, keep the answer concise
Question: just tell me the definition of implicit functions, keep the answer concise
Answer: An implicit function is a relationship between two variables, x and y, expressed as an equation, where y is not explicitly expressed as a function of x.
